In [4]:
import pandas as pd

school_path = 'Resources/schools_complete.csv'
student_path = 'Resources/students_complete.csv'

school_df = pd.read_csv(school_path)
student_df = pd.read_csv(student_path)

full_df = pd.merge(student_df, school_df, how='left', on=['school_name', 'school_name'])

In [5]:
full_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [13]:
total_schools = len(full_df['school_name'].unique())
total_students = len(full_df['Student ID'].unique())
total_budget = full_df.budget.unique().sum()
average_math = (full_df.math_score.sum())/len(full_df.math_score)
average_reading = (full_df.reading_score.sum())/len(full_df.reading_score)
passing_math = (len(full_df.loc[full_df['math_score'] >= 70, :]) / total_students) * 100
passing_reading = (len(full_df.loc[full_df['reading_score'] >= 70, :]) / total_students) * 100

district_summary = pd.DataFrame({'Total Schools': total_schools, 'Total Students': total_students, 'Total Budget': total_budget, 'Average Math Score': average_math, 'Average Reading Score': average_reading, '% Passing Math': passing_math, '% Passing Reading': passing_reading}, index=[0])
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463


In [14]:
full_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [16]:
school_group = full_df.groupby('school_name')

school_group_df = pd.DataFrame()
school_group_df['School Type'] = school_group['type'].first()
school_group_df['Total Students'] = school_group['Student ID'].count()
school_group_df['Total School Budget'] = school_group['budget'].max()
school_group_df['Per Student Budget'] = (school_group_df['Total School Budget'] / school_group_df['Total Students'])
school_group_df['Average Math Score'] = school_group['math_score'].mean()
school_group_df['Average Reading Score'] = school_group['reading_score'].mean()

math_passing_df = full_df.loc[full_df['math_score'] >= 70, :]
math_passing_grouped = math_passing_df.groupby('school_name')
school_group_df['% Passing Math'] = (math_passing_grouped['Student ID'].count() / school_group_df['Total Students']) * 100
# school_group.first()

reading_passing_df = full_df.loc[full_df['reading_score'] >= 70, :]
reading_passing_grouped = reading_passing_df.groupby('school_name')
school_group_df['% Passing Reading'] = (reading_passing_grouped['Student ID'].count() / school_group_df['Total Students']) * 100

school_group_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading
school_name,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965
